In [2]:
import pandas as pd
from pathlib import Path

from posydon.popsyn.binarypopulation import BinaryPopulation
from posydon.binary_evol.binarystar import BinaryStar
from posydon.binary_evol.singlestar import SingleStar
from posydon.popsyn.synthetic_population import Population
from posydon.popsyn.synthetic_population import PopulationRunner
import astropy.units as u

import os
import shutil
from posydon.config import PATH_TO_POSYDON

from POSYDONHRDiagramModule import HR_Diagram

pip install --upgrade "git+https://github.com/PiersonLip/Posydon_HR_Graphing_Script.git@main"


srun -A b1094 -p ciera-std --time=04:00:00 --mem=16G --pty /bin/bash

cd /home/bku2126/b1095/bku2126/UCXBInvestigation/Project/Analysis

conda activate posydon_env_v2.2

hostname

jupyter notebook --port=8383 --no-browser

In [3]:
loadedpop = Population(r'/home/bku2126/b1095/bku2126/UCXBInvestigation/Project/Grids/v2_ucxb/1e+00_Zsun_population.h5')

In [4]:
cols = ['time', 'step_names', 'state', 'event', 'S1_state', 'S2_state', 'S1_mass', 'S2_mass', 'orbital_period']
finCols = [
    'orbital_period_f',
    'eccentricity_f',
    'state_f',

    'S2_state_f',
    'S2_mass_f',
    'S2_log_R_f',
    'S2_log_L_f',

    'S1_state_f',
    'S1_mass_f',
    'S1_log_R_f',
    'S1_log_L_f'

 ]

initCols = [
    'orbital_period_i',
    'eccentricity_i',
    'state_i',

    'S2_state_i',
    'S2_mass_i',
    'S2_log_R_i',

    'S1_state_i',
    'S1_mass_i',
    'S1_log_R_i'
 ]

In [5]:
XrB_Logic_Oneline = "((S2_state_f == 'BH') | (S2_state_f == 'NS')) & ((S1_state_f != 'BH') & (S1_state_f != 'NS')) | ((S1_state_f == 'BH') | (S1_state_f == 'NS')) & ((S2_state_f != 'BH') & (S2_state_f != 'NS')) & ((state_f == 'initial_RLOF') | (state_f == 'RLO2') | (state_f == 'detached')) "

XrB_Logic_History = "((S2_state == 'BH') | (S2_state == 'NS')) & ((S1_state != 'BH') & (S1_state != 'NS')) | ((S1_state == 'BH') | (S1_state == 'NS')) & ((S2_state != 'BH') & (S2_state != 'NS')) & ((state == 'initial_RLOF') | (state == 'RLO2') | (state == 'detached')) "

In [6]:
XrBs = loadedpop.oneline.select(where=XrB_Logic_Oneline)

In [7]:
orbitalPeriodMax = 120 * u.minute
OP_Max_val = orbitalPeriodMax.to(u.day).value
UCXBs = XrBs[XrBs['orbital_period_f'] < OP_Max_val]

In [15]:
UCXBs[UCXBs['state_f'] == 'detached'][finCols]

,orbital_period_f,eccentricity_f,state_f,S2_state_f,S2_mass_f,S2_log_R_f,S2_log_L_f,S1_state_f,S1_mass_f,S1_log_R_f,S1_log_L_f
binary_index,,,,,,,,,,,
858700,0.060739,0.0,detached,WD,0.830677,-2.353343,NaN,NS,1.283187,-4.745687,NaN


In [9]:
UCXB_Indexes = UCXBs.index.to_list()

In [69]:
loadedpop.export_selection(selection=UCXB_Indexes, filename = '1e+00_Zsun_UCXB_subpop.h5', append=True)

posydon-popsyn setup ucxb_grid_params_v_2.ini --job_array=50 --walltime=24:00:00 --partition=ciera-std --account=b1094 --email=piersonlipschultz@gmail.com --mem_per_cpu=12G


bash slurm_submit.sh